# MGMTMSA 403: Optimization
# Assignment 3: Predicting Airbnb Prices
### Elen Sahakyan (UID: 706310696)
### Jane Lee (UID: )


In [29]:
# Importing libraries
import numpy as np  
import pandas as pd
from gurobipy import *
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [30]:
# Importing data
train = pd.read_csv('AirbnbTrain.csv')
train.head()

,latitude,longitude,Entire home,accommodates,bathrooms,bedrooms,beds,cleaning_fee,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,price
0,34.103701,-118.332241,1,13,2.0,3,2,150,2,1,100,1,350
1,34.099484,-118.331645,1,8,2.0,2,4,150,1,11,96,1,190
2,34.104321,-118.329662,1,4,1.0,0,1,55,1,1,80,0,85
3,34.101028,-118.317848,0,2,1.0,1,1,20,1,8,98,0,75
4,34.098292,-118.324980,1,2,1.0,1,1,20,1,11,96,0,130


In [31]:
train.describe()

,latitude,longitude,Entire home,accommodates,bathrooms,bedrooms,beds,cleaning_fee,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,price
count,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000
mean,34.099711,-118.334020,0.818824,3.978824,1.292353,1.118235,2.245882,74.559412,2.815882,36.219412,92.871765,0.605882,144.968235
std,0.004980,0.011726,0.385278,2.292249,0.557046,0.867718,1.829588,52.007940,5.517449,50.783103,8.694679,0.488804,115.219601
min,34.083418,-118.361549,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,20.000000,0.000000,0.000000
25%,34.098312,-118.342887,1.000000,2.000000,1.000000,1.000000,1.000000,40.000000,1.000000,5.000000,90.000000,0.000000,88.000000
50%,34.101492,-118.335496,1.000000,4.000000,1.000000,1.000000,2.000000,70.000000,1.000000,14.000000,95.000000,1.000000,120.000000
75%,34.102889,-118.325845,1.000000,5.000000,1.500000,2.000000,3.000000,99.000000,2.000000,47.000000,98.000000,1.000000,175.000000
max,34.105243,-118.309413,1.000000,16.000000,5.500000,7.000000,24.000000,460.000000,60.000000,352.000000,100.000000,1.000000,2000.000000


In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   latitude              1700 non-null   float64
 1   longitude             1700 non-null   float64
 2   Entire home           1700 non-null   int64  
 3   accommodates          1700 non-null   int64  
 4   bathrooms             1700 non-null   float64
 5   bedrooms              1700 non-null   int64  
 6   beds                  1700 non-null   int64  
 7   cleaning_fee          1700 non-null   int64  
 8   minimum_nights        1700 non-null   int64  
 9   number_of_reviews     1700 non-null   int64  
 10  review_scores_rating  1700 non-null   int64  
 11  instant_bookable      1700 non-null   int64  
 12  price                 1700 non-null   int64  
dtypes: float64(3), int64(10)
memory usage: 172.8 KB


In [33]:
train = train.rename(columns = {'Entire home': 'entire_home'})
train.columns

Index(['latitude', 'longitude', 'entire_home', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'cleaning_fee', 'minimum_nights',
       'number_of_reviews', 'review_scores_rating', 'instant_bookable',
       'price'],
      dtype='object')

In [34]:
test = pd.read_csv('AirbnbTest.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   latitude              699 non-null    float64
 1   longitude             699 non-null    float64
 2   Entire home           699 non-null    int64  
 3   accommodates          699 non-null    int64  
 4   bathrooms             699 non-null    float64
 5   bedrooms              699 non-null    int64  
 6   beds                  699 non-null    int64  
 7   cleaning_fee          699 non-null    int64  
 8   minimum_nights        699 non-null    int64  
 9   number_of_reviews     699 non-null    int64  
 10  review_scores_rating  699 non-null    int64  
 11  instant_bookable      699 non-null    int64  
 12  price                 699 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 71.1 KB


In [35]:
test = test.rename(columns = {'Entire home': 'entire_home'})
test.columns

Index(['latitude', 'longitude', 'entire_home', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'cleaning_fee', 'minimum_nights',
       'number_of_reviews', 'review_scores_rating', 'instant_bookable',
       'price'],
      dtype='object')

**Model 1** Linear regression model on price optimizing for the least absolute deviations.

In [36]:
# Construct a linear program to minimize the absolute deviations on the training dataset
# Construct a 'blank' model
model1 = Model()

# Add decision variables for the intercept and beta estimates
# intercept = model1.addVar(vtype=GRB.CONTINUOUS)
beta_estimates = model1.addVars(len(train.columns)-1, vtype=GRB.CONTINUOUS)

# Add decision variables for the absolute residuals
abs_residuals = model1.addVars(len(train), vtype=GRB.CONTINUOUS)

# Set objective function to minimize the average of absolute residuals
model1.setObjective(quicksum(abs_residuals[i] for i in range(len(train))) / len(train), GRB.MINIMIZE)

# Add constraints for absolute residuals
for i in range(len(train)):
    predicted_price = quicksum(beta_estimates[j] * train.loc[i, train.columns[j]] for j in range(len(beta_estimates)))
    model1.addConstr(abs_residuals[i] >= train.loc[i, 'price'] - predicted_price)
    model1.addConstr(abs_residuals[i] >= predicted_price - train.loc[i, 'price'])

# for i in range(len(train)):
#     predicted_price = intercept + quicksum(beta_estimates[j] * train.loc[i, train.columns[j]] for j in range(len(beta_estimates)))
#     model1.addConstr(abs_residuals[i] == abs(float(train.loc[i, 'price']) - predicted_price))

# Update and optimize the model
model1.update()
model1.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3400 rows, 1712 columns and 41372 nonzeros
Model fingerprint: 0xa0d487df
Coefficient statistics:
  Matrix range     [5e-01, 5e+02]
  Objective range  [6e-04, 6e-04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]
Presolve time: 0.01s
Presolved: 3400 rows, 1712 columns, 41372 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 12
 AA' NZ     : 2.995e+04
 Factor NZ  : 3.260e+04 (roughly 2 MB of memory)
 Factor Ops : 4.141e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.13176381e+03  0.00000000e+00  1.36e+03 0.00e+00  2.

In [37]:
# Print results - beta estimates
print(f'Beta Estimates: Training Dataset\n')
# print(f'Intercept: {intercept.x:.4f}\n')
if model1.status == GRB.OPTIMAL:
    for i in range(len(beta_estimates)):
        print(f'{train.columns[i]}: {beta_estimates[i].x:.4f}')
print(f'\n')
print(f'Error (SAD): {model1.objVal:.4f}')

Beta Estimates: Training Dataset

latitude: 290.2515
longitude: 84.0309
entire_home: 36.7832
accommodates: 9.9368
bathrooms: 31.6943
bedrooms: 19.7040
beds: 0.0000
cleaning_fee: 0.3106
minimum_nights: 0.0000
number_of_reviews: 0.0000
review_scores_rating: 0.2681
instant_bookable: 5.1675


Error (SAD): 36.4262


In [38]:
# Calculate prediction error on test dataset with the optimized beta estimates
beta_values = model1.getAttr('X', beta_estimates)   
test['predicted_price'] = sum(beta_values[j] * test[test.columns[j]] for j in range(len(beta_estimates)))

# Calculate the residuals (absolute difference between actual and predicted prices)
test['residuals'] = abs(test['price'] - test['predicted_price'])

# Calculate the average prediction error
avg_prediction_error = test['residuals'].mean()

# Print results - prediction error (SAD)
print(f'Average Prediction Error of Model 1 on Test Dataset is ${avg_prediction_error:.4f} per night')

Average Prediction Error of Model 1 on Test Dataset is $35.6045 per night


**Model 2** Build a model that predicts Airbnb prices using only the three significantly most important variables.

In [39]:
# Construct a 'blank' model
model2 = Model()

num_predictors = len(train.columns) - 1 
num_vars = 3  

# Add decision variables for the beta estimates
beta_estimates2 = model2.addVars(num_predictors, vtype=GRB.CONTINUOUS)

# Add binary variables for selecting variables
select_vars = model2.addVars(num_predictors, vtype=GRB.BINARY)

# Add decision variables for the absolute residuals
abs_residuals = model2.addVars(len(train), vtype=GRB.CONTINUOUS)

# Set objective function to minimize the average of absolute residuals
model2.setObjective(quicksum(abs_residuals[i] for i in range(len(train))) / len(train), GRB.MINIMIZE)

# Constraint to ensure only three variables are selected
model2.addConstr(quicksum(select_vars[j] for j in range(num_predictors)) == num_vars)

# Add constraints for absolute residuals
for i in range(len(train)):
    predicted_price = quicksum(beta_estimates2[j] * train.loc[i, train.columns[j]] * select_vars[j] for j in range(num_predictors))
    model2.addConstr(abs_residuals[i] >= train.loc[i, 'price'] - predicted_price)
    model2.addConstr(abs_residuals[i] >= predicted_price - train.loc[i, 'price'])

# Update and optimize the model
model2.update()
model2.optimize()


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 1724 columns and 12 nonzeros
Model fingerprint: 0xdd48feb3
Model has 3400 quadratic constraints
Variable types: 1712 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [5e-01, 5e+02]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
  QRHS range       [1e+01, 2e+03]
Presolve added 824 rows and 410 columns
Presolve time: 0.03s
Presolved: 3421 rows, 2170 columns, 32958 nonzeros
Presolved model has 24 SOS constraint(s)
Variable types: 2146 continuous, 24 integer (24 binary)
Found heuristic solution: objective 61.0111721
Found heuristic solution: objective 55.2795580

Root relaxation: objective 3.642625e+01, 2119 iterations, 0.12 seconds

In [40]:
# Print results - selected variables and their coefficients
print(f'Variables: Training Dataset\n')
if model2.status == GRB.OPTIMAL:
    variables = [(train.columns[i], beta_estimates2[i].x) for i in range(num_predictors) if beta_estimates2[i].x != 0]
    variables.sort(key=lambda x: x[1], reverse=True)
    for var, value in variables:
        print(f'{var}: {value:.2f}')
print(f'\n')
print(f'Error (SAD) on Training Model: {model2.objVal:.2f}')

Variables: Training Dataset

entire_home: 52.00
bedrooms: 32.00
accommodates: 14.00


Error (SAD) on Training Model: 38.34


In [41]:
# Calculate prediction error on test dataset with the selected variables and their coefficients
beta_values2 = model2.getAttr('X', beta_estimates2)   
test['predicted_price'] = sum(beta_values2[j] * test[test.columns[j]] for j in range(len(beta_estimates2)))

# Calculate the residuals (absolute difference between actual and predicted prices)
test['residuals'] = abs(test['price'] - test['predicted_price'])

# Calculate the average prediction error
avg_prediction_error = test['residuals'].mean()

# Print results - prediction error (SAD)
print(f'Average Prediction Error of Model 2 on Test Dataset is ${avg_prediction_error:.4f} per night')

Average Prediction Error of Model 2 on Test Dataset is $37.7368 per night


**Model 3** Build a model that predicts Airbnb prices using number of beds ('beds') as a predictor, and chooses two other significantly important variables.

In [44]:
# Construct a 'blank' model
model3 = Model()

num_predictors3 = len(train.columns) - 1  
num_vars3 = 2 

# Add decision variables for the beta estimates
beta_estimates3 = model3.addVars(num_predictors3, vtype=GRB.CONTINUOUS)

# Add binary variables for selecting variables
select_vars3 = model3.addVars(num_predictors3, vtype=GRB.BINARY)

# Include 'beds' in the model
beds_index = train.columns.tolist().index('beds')
select_vars3[beds_index].lb = 1

# Add decision variables for the absolute residuals
abs_residuals3 = model3.addVars(len(train), vtype=GRB.CONTINUOUS)

# Set objective function to minimize the average of absolute residuals
model3.setObjective(quicksum(abs_residuals3[i] for i in range(len(train))) / len(train), GRB.MINIMIZE)

# Constraint to ensure only two additional variables are selected
model3.addConstr(quicksum(select_vars3[j] for j in range(num_predictors3)) == num_vars3 + 1)

# Add constraints for absolute residuals
for i in range(len(train)):
    predicted_price = quicksum(beta_estimates3[j] * train.loc[i, train.columns[j]] * select_vars3[j] for j in range(num_predictors3))
    model3.addConstr(abs_residuals3[i] >= train.loc[i, 'price'] - predicted_price)
    model3.addConstr(abs_residuals3[i] >= predicted_price - train.loc[i, 'price'])

# Update and optimize the model
model3.update()
model3.optimize()


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 1724 columns and 12 nonzeros
Model fingerprint: 0xd615c986
Model has 3400 quadratic constraints
Variable types: 1712 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [5e-01, 5e+02]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
  QRHS range       [1e+01, 2e+03]
Presolve added 824 rows and 409 columns
Presolve time: 0.05s
Presolved: 3419 rows, 2166 columns, 32952 nonzeros
Presolved model has 22 SOS constraint(s)
Variable types: 2144 continuous, 22 integer (22 binary)
Found heuristic solution: objective 48.5975270

Root relaxation: objective 3.642625e+01, 2529 iterations, 0.11 seconds (0.43 work units)

    Nodes    |    Current N

In [45]:
# Print results - selected variables and their coefficients
print(f'Variables: Training Dataset\n')
if model3.status == GRB.OPTIMAL:
    variables = [(train.columns[i], beta_estimates3[i].x) for i in range(num_predictors3) if beta_estimates3[i].x != 0]
    variables.sort(key=lambda x: x[1], reverse=True)
    for var, value in variables:
        print(f'{var}: {value:.2f}')
print(f'\n')
print(f'Error (SAD) on Training Model: {model3.objVal:.2f}')

Variables: Training Dataset

entire_home: 67.88
bedrooms: 47.37
beds: 12.13


Error (SAD) on Training Model: 40.07


In [46]:
# Calculate prediction error on test dataset with the selected variables and their coefficients
beta_values3 = model3.getAttr('X', beta_estimates3)   
test['predicted_price'] = sum(beta_values3[j] * test[test.columns[j]] for j in range(len(beta_estimates3)))

# Calculate the residuals (absolute difference between actual and predicted prices)
test['residuals'] = abs(test['price'] - test['predicted_price'])

# Calculate the average prediction error
avg_prediction_error3 = test['residuals'].mean()

# Print results - prediction error (SAD)
print(f'Average Prediction Error of Model 3 on Test Dataset is ${avg_prediction_error3:.4f} per night')

Average Prediction Error of Model 3 on Test Dataset is $38.5996 per night


Including 'beds' as one of the variables leads to the exclusion of 'accommodates' from the model, while 'entire_home' and 'bedrooms' remain intact. Since the model is optimized to minimize the Sum of Absolute Deviations (SAD), we can infer that 'entire_home' and 'bedrooms' are the two most significant variables in achieving the Error minimization.

Although there is not enough evidence provided, it is worth noting that the variables 'bedrooms', 'beds', and 'accommodates' are measures of the size of the Airbnb accommodation and may exhibit some correlation. By including 'beds', the model accounts for some of the variation that the other variables might otherwise capture.